In [1]:
import pandas as pd
import numpy


#Abrir base de datos
file_path = r'C:\Users\subce\Downloads\BD2.txt'
data = pd.read_csv(filepath_or_buffer = file_path, sep = ";")
datos = numpy.asarray(data)

In [28]:
%%time
#2 valores MinPTS, Eps
import numpy

poblacion_nueva = []
num_weights = 2 
sol_per_pob = 10 #Poblacion por generación
pop_size = (sol_per_pob,num_weights) 


#Intervalos para MinPts Eps (Parametros DBscan)
MinPts = numpy.random.randint(low=6, high=20, size= sol_per_pob) 
Eps = numpy.random.randint(low=1, high=5, size= sol_per_pob)

for x in range(0, sol_per_pob):
    poblacion_nueva.append([MinPts[x], Eps[x]])

num_generations = 10 #Las generaciones nuevas
num_padres_mating = 4 #Padres que se van a reproducir 

for generation in range(num_generations):
    print("Generación : ", generation)
    # Medir el fitnes de la población
    
    fitness = cal_pob_fitness(poblacion_nueva)

    # Seleccionar los mejores padres
    
    padres = selec_pob_reproducir(poblacion_nueva, fitness, num_padres_mating)

    # Generar la nueva generación a partir de el cruce simple
    
    offspring_crossover = crossover(padres, tamano_cruce = (pop_size[0]-padres.shape[0], num_weights))

    # Añadir mutación
    offspring_mutation = mutacion(offspring_crossover)

    # Crear la nueva población
    poblacion_nueva[0:padres.shape[0] :] = padres
    poblacion_nueva[padres.shape[0]: :] = offspring_mutation

    # Mejor resultado de la iteración
    print("Mejor resultado : ", numpy.max(numpy.array(fitness)))


fitness = cal_pob_fitness(poblacion_nueva)

best_match_idx = numpy.argmax(fitness)
print("Mejor solución : ", poblacion_nueva[best_match_idx])
print("Fitness de la mejor solución : ", fitness[best_match_idx])

Generación :  0
Mejor resultado :  0.6240081846200443
Generación :  1
Mejor resultado :  1.1870406926147015
Generación :  2
Mejor resultado :  1.1870406926147015
Generación :  3
Mejor resultado :  1.1870406926147015
Generación :  4
Mejor resultado :  1.254992043111912
Generación :  5
Mejor resultado :  1.254992043111912
Generación :  6
Mejor resultado :  1.254992043111912
Generación :  7
Mejor resultado :  1.254992043111912
Generación :  8
Mejor resultado :  1.254992043111912
Generación :  9
Mejor resultado :  1.3601320460935897
Mejor solución :  [13337.    25.]
Fitness de la mejor solución :  1.3640019728198534
Wall time: 54min 33s


In [22]:
from sklearn import metrics
from sklearn.cluster import DBSCAN
from collections import Counter, defaultdict
import numpy as np 

def cal_pob_fitness(poblacion_nueva):
    fitnes = 0
    calinski = []
    davies   = []
    for individuo in poblacion_nueva:
        clustering = DBSCAN(min_samples=individuo[0] , eps=individuo[1]).fit(datos)
        labels = clustering.labels_
        n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
        n_noise_ = list(labels).count(-1)
        
        if(n_clusters_ >= 1):
            calinski.append(metrics.calinski_harabasz_score(datos, labels))
            davies.append(1 - metrics.davies_bouldin_score(datos, labels))
        else:
            calinski.append(0)
            davies.append(0)
        
    calisnky_arr = np.array(calinski)
    calisnky_norm = (calisnky_arr - calisnky_arr.min())/ (calisnky_arr.max() - calisnky_arr.min())
    
    fitness = []
    
    for x in range(0, len(calisnky_norm)):
        fitness.append(calisnky_norm[x] + davies[x])

    return(fitness)


In [3]:
def selec_pob_reproducir(pob, fitness, num_padres):

    fitness = np.array(fitness)
    pob = np.array(pob)
    padres = numpy.empty((num_padres, pob.shape[1]))

    for parent_num in range(num_padres):

        max_fitness_idx = numpy.where(fitness == numpy.max(fitness))

        max_fitness_idx = max_fitness_idx[0][0]

        padres[parent_num, :] = pob[max_fitness_idx, :]

        fitness[max_fitness_idx] = -99999999999

    return padres

In [4]:
def crossover(padres, tamano_cruce):
    
    decendientes = numpy.empty(tamano_cruce)
    crossover_point = numpy.uint8(tamano_cruce[1]/2) 
     for k in range(tamano_cruce[0]):
         parent1_idx = k%padres.shape[0]
         parent2_idx = (k+1)%padres.shape[0]
         decendientes[k, 0:crossover_point] = padres[parent1_idx, 0:crossover_point]
         decendientes[k, crossover_point:] = padres[parent2_idx, crossover_point:]
     return decendientes

In [21]:
import random
def mutacion(nueva_descendencia):
    indice_mutacion = random.randint(0, len(nueva_descendencia) -1)
    Eps =  random.randint(1, 30)
    MinPts =  random.randint(6, 50000)
    nueva_descendencia[indice_mutacion][0] = int(MinPts)
    nueva_descendencia[indice_mutacion][1] = int(Eps)
    
    return nueva_descendencia